# Graph Serialization Example

This notebook demonstrates how to:
1. Create a graph
2. Serialize it to JSON
3. Save it to a file
4. Load it back from the file

## Setup

In [ ]:
import tempfile
import os
from pathlib import Path

from netrun_core import (
    Graph, Node, Edge, EdgeRef, Port, PortRef, PortType, PortSlotSpec,
    PortState, SalvoCondition, SalvoConditionTerm,
)

## Create a Graph

Let's create a simple diamond-shaped graph: A → B, C → D

In [ ]:
# Node A: Source with two outputs
node_a = Node(
    name="A",
    out_ports={
        "out1": Port(PortSlotSpec.infinite()),
        "out2": Port(PortSlotSpec.infinite()),
    },
)

# Nodes B and C: Middle nodes
node_b = Node(
    name="B",
    in_ports={"in": Port(PortSlotSpec.infinite())},
    out_ports={"out": Port(PortSlotSpec.infinite())},
    in_salvo_conditions={
        "default": SalvoCondition(
            max_salvos=1,
            ports=["in"],
            term=SalvoConditionTerm.port("in", PortState.non_empty()),
        ),
    },
)

node_c = Node(
    name="C",
    in_ports={"in": Port(PortSlotSpec.infinite())},
    out_ports={"out": Port(PortSlotSpec.infinite())},
    in_salvo_conditions={
        "default": SalvoCondition(
            max_salvos=1,
            ports=["in"],
            term=SalvoConditionTerm.port("in", PortState.non_empty()),
        ),
    },
)

# Node D: Sink with two inputs (synchronization)
node_d = Node(
    name="D",
    in_ports={
        "in1": Port(PortSlotSpec.infinite()),
        "in2": Port(PortSlotSpec.infinite()),
    },
    in_salvo_conditions={
        "sync": SalvoCondition(
            max_salvos=1,
            ports=["in1", "in2"],
            term=SalvoConditionTerm.and_([
                SalvoConditionTerm.port("in1", PortState.non_empty()),
                SalvoConditionTerm.port("in2", PortState.non_empty()),
            ]),
        ),
    },
)

print("Created nodes: A, B, C, D")

In [ ]:
# Create edges: A -> B, C and B, C -> D
def make_edge(src_node: str, src_port: str, tgt_node: str, tgt_port: str):
    return (
        EdgeRef(
            PortRef(src_node, PortType.Output, src_port),
            PortRef(tgt_node, PortType.Input, tgt_port),
        ),
        Edge(),
    )

edges = [
    make_edge("A", "out1", "B", "in"),
    make_edge("A", "out2", "C", "in"),
    make_edge("B", "out", "D", "in1"),
    make_edge("C", "out", "D", "in2"),
]

print(f"Created {len(edges)} edges")

In [ ]:
# Build the graph
graph = Graph([node_a, node_b, node_c, node_d], edges)

# Validate
errors = graph.validate()
if errors:
    print(f"Validation errors: {errors}")
else:
    print(f"Graph created successfully: {graph}")

## Serialize to JSON

The `to_json()` method converts the graph to a JSON string.

In [ ]:
json_str = graph.to_json()

print(f"JSON size: {len(json_str)} bytes")
print()
print(json_str)

## Save to a Temporary File

In [ ]:
# Create a temporary directory
tmp_dir = tempfile.mkdtemp(prefix="netrun_")
file_path = Path(tmp_dir) / "graph.json"

# Write the JSON to file
file_path.write_text(json_str)

print(f"Saved graph to: {file_path}")
print(f"File size: {file_path.stat().st_size} bytes")

## Read and Display the File Contents

In [ ]:
# Read the file back
file_contents = file_path.read_text()

print(f"=== Contents of {file_path.name} ===")
print(file_contents)

## Load the Graph from File

Use `Graph.from_json()` to deserialize the JSON back into a Graph object.

In [ ]:
# Load from the file
loaded_json = file_path.read_text()
loaded_graph = Graph.from_json(loaded_json)

print(f"Loaded graph: {loaded_graph}")

## Verify the Loaded Graph

In [ ]:
# Check nodes
print("Nodes in loaded graph:")
for name, node in loaded_graph.nodes().items():
    in_ports = list(node.in_ports.keys())
    out_ports = list(node.out_ports.keys())
    print(f"  {name}: in_ports={in_ports}, out_ports={out_ports}")

In [ ]:
# Check edges
print("Edges in loaded graph:")
for edge_ref in loaded_graph.edges().keys():
    print(f"  {edge_ref.source.node_name}.{edge_ref.source.port_name} -> {edge_ref.target.node_name}.{edge_ref.target.port_name}")

In [ ]:
# Validate the loaded graph
errors = loaded_graph.validate()
if errors:
    print(f"Validation errors: {errors}")
else:
    print("Loaded graph is valid!")

## Cleanup

In [ ]:
# Clean up the temporary file
import shutil
shutil.rmtree(tmp_dir)
print(f"Cleaned up: {tmp_dir}")

## Summary

We demonstrated:
1. Creating a diamond graph with synchronization logic
2. Serializing it to JSON with `graph.to_json()`
3. Saving to a file
4. Loading back with `Graph.from_json()`
5. Verifying the loaded graph matches the original

This allows you to save graph topologies for later use, share them between systems, or version control them.